Code for processing the data that I have, aligning labels for decoding
Should import with py-scipystack
For working with Sherlock's interface, need to import modules. Need to have specific module loading for it to work otherwise go to the input.sh and run those commands in this notebook. 
(here is the sequence that works: py-scipystack math py-autograd py-pytorch cuda praat )

What features do we want?

We have:
1. phonemes per time
2. silence
3. F0 pitch per time (individually on Praat)
4. dB per time (individually audio on Praat with line separation)

TODO: (see more planning doc for more scripts)
1. Volume (db); this is described as "intensity" on praat
2. pitch characteristics (spectrogram?) (can get pitch Hz from the github script on praat: https://github.com/lennes/pitch-distributions/blob/master/collectPitchSamplesFromCorpus.praat)
   1. idea here: may be able to use strongest power frequencies or extract from Praat (corroborate from Praat)
   2. corroborate with MFA phonemes that are vocal

Decoder characteristics:
1. silence vs sound
2. speech envelope (volume)
3. maybe discretize pitch first into a few categories
   -for pitch, probably need to normalize? and do relative pitch from average or something

Create jupyter instance
check dependencies
figure out how to export Praat alignments and textgrids
parse those into bins
align audios with the cues that we have
align labels with cues that we have

other question: do we need to align at all? can bypass and use bin time of 20ms to align. 


Task audio, mic audio

Use 'trial_start_nsp_analog_time', 'trial_end_nsp_analog_time'
To align with nap time

Check with last trial end in ns5

Audio at 30KHz

Can play audio back from ns5

20ms binsize

maybe also get the neural PSTHs around word onset?

#delay time start
#check if there is a delay with the ns5 data (use blackrock documentation)
#load ns5 data--> one isneural, another is analog, other is digital
#this has audio stream. check to make sure audio stream is aligned with what we expect with the 20ms bin timing

In [ ]:
#assume that labels are ready in 20ms time intervals
#assume we know alignment thru redis clock#tet###fasdffasd 

testin2g2


In [2]:
import math

In [1]:
import numpy as np

In [2]:
import scipy.io as sio

ts_filepath = '/home/groups/henderj/rzwang/exp_data/prosody/t12-02-20-2025/20250220_082208_(1).mat'
# 5. Load redis MATLAB file with nsp timestamp information.
mat_data = sio.loadmat(ts_filepath)

In [3]:
print(mat_data.keys())

['block_number', 'block_start_time', 'all_candidate_sentence_acoustic_scores', 'graph_name', 'confirmation_epoch_start_redis_time', 'trial_start_nsp_analog_time', 'trial_end_nsp_analog_time', 'delay_duration_ms', 'all_candidate_sentence_oldlm_scores', 'cue3_start_nsp_analog_time', 'decoder_signal', 'all_candidate_sentence_newlm_scores', 'binned_neural_spike_band_power', 'session_description', 'confirmation_epoch_start_nsp_analog_time', 'binned_neural_redis_clock', 'binned_neural_threshold_crossings', 'task_type', 'participant', 'trial_start_redis_time', 'corrected_with_candidate', 'microphone_nsp_time', 'correct_status', 'inter_trial_duration_ms', 'trial_end_nsp_neural_time', 'cue', 'all_candidate_sentences', 'block_description', 'ngram_decoder_partial_output', '__version__', 'word_count_mismatch', 'norm_channel_stds', 'trial_accuracy_confirmation', 'binned_neural_nsp_timestamp', 'cue3', 'cue2_start_nsp_neural_time', 'trial_end_redis_time', 'cue3_start_redis_time', 'cue2_start_redis_ti

In [4]:
#mat_data probing. Let's go on redis clock, rather than nsp time (since bins are currently in redis time)
#bins should be 20ms bins
print(mat_data['binned_neural_redis_clock']) #starts at 1740068451564, ends 1740068502478. Probably the end of bins? or the start of bins
#todo: check whether bin times are start or end boundaries
print(mat_data['binned_neural_redis_clock'].shape) #1x2547 long
print(mat_data['block_start_time']) #1740068439
print(mat_data['binned_neural_spike_band_power'].shape) #array of floats, 2547x256 (time x array)
print(mat_data['binned_neural_threshold_crossings'].shape) #binary array, 2547x256 (time x array)
print(mat_data['task_type'])
print(mat_data['cue']) #array of cue lines (audio file, corresponding transcript, number of words. can use this to match trials with labels/audio files, potentially)
#side note: I also have block .txt files that have this information
print(mat_data['binned_neural_nsp_timestamp'])
print(mat_data['go_cue_nsp_neural_time'])
print(mat_data['go_cue_redis_time']) #go cues for each trial, on the redis clock
print(mat_data['trial_start_redis_time']) #trial start times on the redis clock
print(mat_data['trial_end_redis_time']) #trial end times on the redis clock


#I think it's the binned_neural_redis_clock

[[1740068451564 1740068451584 1740068451604 ... 1740068502438
  1740068502458 1740068502478]]
(1, 2547)
[[1740068439]]
(2547, 256)
(2547, 256)
[]
[u"scaled_100wpm_POD0000000240_S0000228.wav;AND IF IT'S MICHAEL;4;1.650000000000091;podcast;2.42;1.82;1.45                  "
 u'scaled_100wpm_POD0000000320_S0000205.wav;I CAN PLAY THE MOVIE;5;1.5399999999999636;podcast;3.25;2.44;1.95                '
 u"scaled_100wpm_POD0000000416_S0000055.wav;WHO ARE COGNIZANT OF WHAT'S HAPPENING;6;2.7299999999999613;podcast;2.2;1.65;1.32"
 u'scaled_100wpm_AUD0000001043_S0000823.wav;NONE FOR ME;3;1.2300000000000182;audiobook;2.44;1.83;1.46                       '
 u'scaled_100wpm_POD0000000352_S0000314.wav;MONDAY BEGAN LIKE A NORMAL DAY;6;1.8199999999999363;podcast;3.3;2.47;1.98       ']
[[1.72667492e+18 1.72667492e+18 1.72667492e+18 ... 1.72667497e+18
  1.72667497e+18 1.72667497e+18]]
[[1.72667492e+18 1.72667494e+18 1.72667495e+18 1.72667495e+18
  1.72667496e+18]]
[[1.74006846e+12 1.74006847e+12 1.74006848

In [5]:
block_start = mat_data['block_start_time'][0][0]  # 1740068439

bin_times = mat_data['binned_neural_redis_clock'][0]  # 1x2547 array

# Bins appear to be 20ms (difference between consecutive times is 20)
# Trial-specific timing

trial_starts = mat_data['trial_start_redis_time'][0]
trial_ends = mat_data['trial_end_redis_time'][0]
go_cues = mat_data['go_cue_redis_time'][0]
cues = mat_data['cue']  # Contains audio filename, transcript, word count, etc.

In [ ]:
def mat_to_npy(mat_file_path, output_dir):
    mat_data = sio.loadmat(mat_file_path)
    np.save(f"{output_dir}/spike_band_power.npy", mat_data['binned_neural_spike_band_power'])
    np.save(f"{output_dir}/threshold_crossings.npy",mat_data['binned_neural_threshold_crossings'])
    np.save(f"{output_dir}/bin_times.npy", mat_data['binned_neural_redis_clock'][0])
    np.save(f"{output_dir}/go_times.npy", mat_data['go_cue_redis_time'][0])
    np.save(f"{output_dir}/trial_starts.npy", mat_data['trial_start_redis_time'][0])
    np.save(f"{output_dir}/trial_ends.npy", mat_data['trial_start_redis_time'][0])
    np.save(f"{output_dir}/cues.npy", mat_data['cue'][])

In [2]:
import scipy.io as sio
import numpy as np
import os


#TODO: add the theshold crossings, as this code only has the neural spike band power
def process_all_blocks(exp_dir, label_dir, output_dir, footer):
    """
    Process all blocks in directory and save aligned data as .npy files. Should overwrite existing label files in the folder (if any)
    Parameters:
    -----------
    exp_dir : str
        Directory containing .mat files with the neural data
    label_dir : str
        Directory containing label files (should be .npy). should correspond to the 20ms bins of the neural data
    output_dir : str
        Directory to save processed .npy files
    footer : str
        file label footer (vs .wav file)
        for example, if a.wav has labels a_lbl.npy, this string would be "_lbl.npy"
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    all_neural_data = []
    all_neural_data_tc = []
    all_labels = []
    trial_info = []  # Store metadata about each trial
    
    #Process each .mat file in directory
    mat_files = [f for f in os.listdir(exp_dir) if f.endswith('.mat')]
    for i, mat_file in enumerate(mat_files):
#         print(f"Processing block {i+1}/{len(mat_files)}: {mat_file}")
        print("Processing block {}/{}: {}".format(i + 1, len(mat_files), mat_file))
        # Load mat file
        mat_data = sio.loadmat(os.path.join(exp_dir, mat_file))
        
        # Get block data
        neural_data = mat_data['binned_neural_spike_band_power'] #we should also
        #include the binned_neural_threshold_crossings here?
        neural_data_tc = mat_data['binned_neural_threshold_crossings']
        bin_times = mat_data['binned_neural_redis_clock'][0]
        go_cues = mat_data['go_cue_redis_time'][0]
        trial_ends = mat_data['trial_end_redis_time'][0]
        
        #Process each trial in block
        for trial_idx, (cue_line, go_cue, trial_end) in enumerate(zip(mat_data['cue'], go_cues, trial_ends)):
            #Parse audio filename
            audio_file = cue_line.split(';')[0]
            
            # Load corresponding labels
            label_file = os.path.join(label_dir, audio_file.replace('.wav', footer)) #TODO: change this if doing a different label type
            if not os.path.exists(label_file):
#                 print(f"Warning: Missing label file for {audio_file}")
                print("Warning: Missing label file for {}".format(audio_file))
                continue
                
            labels = np.load(label_file)
            
            # Find trial boundaries
            trial_start_idx = np.searchsorted(bin_times, go_cue)
            trial_end_idx = np.searchsorted(bin_times, trial_end)
            
            # Extract trial data
            trial_neural = neural_data[trial_start_idx:trial_end_idx]
            trial_neural_tc = neural_data_tc[trial_start_idx:trial_end_idx]
            
            # Align lengths
            n_neural_bins = trial_neural.shape[0] #should be the same for tc and sbp
            n_label_bins = len(labels)
            
            if n_neural_bins > n_label_bins:
                labels = np.pad(labels, (0, n_neural_bins - n_label_bins),
                              mode='constant', constant_values=1)
            else:
                labels = labels[:n_neural_bins]
                trial_neural = trial_neural[:n_label_bins]
                trial_neural_tc = trial_neural_tc[:n_label_bins]
            
            # Store aligned data
            all_neural_data.append(trial_neural)
            all_neural_data_tc.append(trial_neural_tc)
            all_labels.append(labels)
            
            # Store trial metadata
            trial_info.append({
                'block_file': mat_file,
                'audio_file': audio_file,
                'trial_idx': trial_idx,
                'n_bins': len(labels)
            })
    
    # Convert to arrays and save
    neural_data_array = np.concatenate(all_neural_data, axis=0)
    neural_data_array_tc = np.concatenate(all_neural_data_tc, axis=0)
    labels_array = np.concatenate(all_labels)
    
#     np.save(f"{output_dir}/neural_data.npy", neural_data_array)
#     np.save(f"{output_dir}/labels.npy", labels_array)
#     np.save(f"{output_dir}/trial_info.npy", trial_info)
    
#     print(f"Processed {len(mat_files)} blocks, {len(trial_info)} trials")
#     print(f"Total timepoints: {len(labels_array)}")
    np.save("{}/neural_data_sbp.npy".format(output_dir), neural_data_array) #spike band power 
    np.save("{}/neural_data_tc.npy".format(output_dir), neural_data_array_tc) #spike band power 
    np.save("{}/labels.npy".format(output_dir), labels_array)
    np.save("{}/trial_info.npy".format(output_dir), trial_info)
    
    print("Processed {} blocks, {} trials".format(len(mat_files), len(trial_info)))
    print("Total timepoints: {}".format(len(labels_array)))
    return neural_data_array



2.7.13 (default, Apr 27 2017, 14:19:21) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-11)]
Lmod Warning: 
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: cudnn/7.0.5
(required by: py-pytorch/0.3.0_py27), python/2.7.13 (required by:
py-scipystack/1.0_py27, py-pytorch/0.3.0_py27)
-------------------------------------------------------------------------------




The following have been reloaded with a version change:
  1) libressl/2.5.3 => libressl/3.2.1     3) sqlite/3.18.0 => sqlite/3.44.2
  2) python/2.7.13 => python/3.9.0

Lmod Warning: 
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: cudnn/6.0 (required
by: torch/20180202), python/3.9.0 (required by: py-jupyter/1.0.0_py39,
py-numpy/1.20.3_py39)
-------------------------------------------------------------------------------




The following have been reloaded wi

In [3]:
label_dir = "/home/groups/henderj/rzwang/labels"
exp_dir = "/home/groups/henderj/rzwang/blocks/t12-02-20-2025"
output_dir = "/home/groups/henderj/rzwang/processed_data"

process_all_blocks(exp_dir, label_dir, output_dir, "_silence.npy")

Processing block 1/14: 20250220_092958_(12).mat
Processing block 2/14: 20250220_094931_(15).mat
Processing block 3/14: 20250220_084811_(5).mat
Processing block 4/14: 20250220_091803_(10).mat
Processing block 5/14: 20250220_084201_(4).mat
Processing block 6/14: 20250220_083525_(3).mat
Processing block 7/14: 20250220_085412_(6).mat
Processing block 8/14: 20250220_082717_(2).mat
Processing block 9/14: 20250220_093742_(13).mat
Processing block 10/14: 20250220_091212_(9).mat
Processing block 11/14: 20250220_090001_(7).mat
Processing block 12/14: 20250220_092404_(11).mat
Processing block 13/14: 20250220_094329_(14).mat
Processing block 14/14: 20250220_090631_(8).mat
Processed 14 blocks, 280 trials
Total timepoints: 46171


array([[151.12434 , 116.256424,  80.161865, ...,  99.960754,  49.764416,
         55.563038],
       [115.588844,  90.225296, 101.437965, ...,  82.24857 ,  50.258266,
         52.168068],
       [ 87.49782 ,  87.25968 , 101.88918 , ..., 179.96828 ,  49.59959 ,
         54.988564],
       ...,
       [105.395996,  92.8964  ,  94.88707 , ...,  86.77444 ,  40.54669 ,
         63.69227 ],
       [105.838135,  94.94759 ,  96.397606, ..., 100.22508 ,  46.997364,
         54.956276],
       [105.87013 ,  87.94528 , 115.41392 , ..., 111.47727 ,  47.287506,
         54.489246]], dtype=float32)

Decoder below didn't work, so moved the model to a .py and ran from there. 

In [4]:
#pilot of the model

import torch
import numpy as np
import torch.optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
# from torchsummary import summary
from scipy import stats

# Function to create windowed data
def create_windows(data, window_size, stride=1):
    """
    Create windowed data for both neural data and labels

    Parameters:
    -----------
    data : np.ndarray
        Neural data (n_timepoints, n_channels)
    window_size : int
        Number of time bins in each window
    stride : int
        Number of bins to stride between windows

    Returns:
    --------
    windowed_data : np.ndarray
        Windowed data (n_windows, n_channels, window_size)
    """

    n_samples = data.shape[0]
    n_windows = ((n_samples - window_size) // stride) + 1

    windows = np.zeros((n_windows, data.shape[1], window_size))

    for i in range(n_windows):
        start_idx = i * stride
        end_idx = start_idx + window_size
        # Transpose to get (n_channels, window_size)
        windows[i] = data[start_idx:end_idx].T

    return windows


# Training script modifications
def train_decoder(neural_data, labels, window_size=5, stride=1):
    # Create windowed data
    X = create_windows(neural_data, window_size, stride)

    # For labels, take the mode within each window
    y = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        start_idx = i * stride
        end_idx = start_idx + window_size
        y[i] = np.mean(labels[start_idx:end_idx])

    #Convert to PyTorch tensors
    X = torch.FloatTensor(X)
    y = torch.FloatTensor(y)

    #Create model and train
    model = CNNSilenceDecoder(n_channels=neural_data.shape[1],
                             window_size=window_size)

    #now train the decoder



class CNNSilenceDecoder(nn.Module):
    def __init__(self, n_channels=256, window_size=5):
        super(CNNSilenceDecoder, self).__init__()
        self.window_size = window_size
        
        self.conv = nn.Sequential(
            nn.Conv1d(n_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(32 * window_size, 1),
            nn.Sigmoid() #output sigmoid for binary guess
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

def train_decoder(neural_data, labels, window_size=5, stride=1, batch_size=32, n_epochs=10, learning_rate=0.001):
    # Create windowed data
    X = create_windows(neural_data, window_size, stride)
    
    #For labels, take the mode within each window
    y = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        start_idx = i * stride
        end_idx = start_idx + window_size
        y[i] = round(np.mean(labels[start_idx:end_idx])) #round should give mode for binary
    
    # Split into train/test sets (80/20 split)
    n_samples = len(X)
    n_train = int(0.8 * n_samples)
    indices = np.random.permutation(n_samples)
    
    train_indices = indices[:n_train]
    test_indices = indices[n_train:]
    
    X_train = torch.FloatTensor(X[train_indices])
    y_train = torch.FloatTensor(y[train_indices])
    X_test = torch.FloatTensor(X[test_indices])
    y_test = torch.FloatTensor(y[test_indices])
    
    #Create data loaders
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train.unsqueeze(1))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    #Initialize model and optimizer
    model = CNNSilenceDecoder(n_channels=neural_data.shape[1], window_size=window_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #TODO: do we want to use Adam
    criterion = nn.BCELoss() #TODO: change this loss function from binary cross entrop
    
    # Training loop
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Evaluate on test set
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test.unsqueeze(1))
            test_preds = (test_outputs > 0.5).float()
            accuracy = (test_preds == y_test.unsqueeze(1)).float().mean()
        
        print("Epoch {}/{}: Train Loss = {:.4f}, Test Loss = {:.4f}, Test Accuracy = {:.4f}".format(
            epoch+1, n_epochs, total_loss/len(train_loader), test_loss.item(), accuracy.item()))
    
    return model

neural_data = np.load("/home/groups/henderj/rzwang/processed_data/neural_data.npy")
labels = np.load("/home/groups/henderj/rzwang/processed_data/labels.npy")

# Train the model
window_size = 5  # 100ms window (assuming 20ms bins)
stride = 1
model = train_decoder(neural_data, labels, 
                     window_size=window_size, 
                     stride=stride,
                     batch_size=16,
                     n_epochs=3,
                     learning_rate=0.001)

# Save the trained model
torch.save(model.state_dict(), "/home/groups/henderj/rzwang/decoders/silence_decoder.pt")

def evaluate_model(model, neural_data, labels, window_size=5, stride=1):
    model.eval()
    X = create_windows(neural_data, window_size, stride)
    y = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        start_idx = i * stride
        end_idx = start_idx + window_size
        y[i] = np.mean(labels[start_idx:end_idx])
    
    X = torch.FloatTensor(X)
    y = torch.FloatTensor(y)
    
    with torch.no_grad():
        outputs = model(X)
        preds = (outputs > 0.5).float()
        accuracy = (preds.squeeze() == y).float().mean()
    
    return accuracy.item()

TypeError: argument 0 is not a Variable